In [38]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import sklearn


original_df = pd.read_csv('top_10000_popular_movies_tmdb.csv')


In [39]:
# Load the model
model_BAAI = SentenceTransformer('BAAI/bge-small-en')
df = pd.read_csv('top_10000_popular_movies_tmdb.csv')

# Define a function to generate embeddings for a single feature column
df["overview_embedding"] = df["overview"].apply(lambda x: model_BAAI.encode(str(x)))


In [40]:
query = "what is the Mario movie about?"
k = 3
similarity_dict = {}
query_embedding = model_BAAI.encode(query)

for index, row in df.iterrows():
    cosine_sim = sklearn.metrics.pairwise.cosine_similarity([row["overview_embedding"]], [query_embedding])[0][0]
    similarity_dict[row["overview"]] = cosine_sim

top_k_overviews = sorted(similarity_dict.items(), key=lambda item: item[1], reverse=True)[:k]

# print(f"Top {k} overviews with the highest cosine similarities:")
# for overview, similarity in top_k_overviews:
#     print(f"Overview: {overview}")
#     print(f"Cosine similarity: {similarity}\n")

instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question "{query}". Don't make up any new information:
{'\n'.join([f' - {overview}' for overview, similarity in top_k_overviews])}'''

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_flan = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

inputs = tokenizer(instruction_prompt, return_tensors="pt")
outputs = model_flan.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['A small animated movie from Marvel and IIRC Blur Studios.']
